In [9]:
from pathlib import Path
import sys

root = Path.cwd().parent
sys.path.append(str(root))

root

PosixPath('/workspace/tasks/financial_ner')

In [10]:
import json

def load_jsonl(path):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    return data

path = root / "data" / "dataset" / "Financial-NER-NLP_dev.jsonl"
val_data = load_jsonl(path)

In [11]:
val_data[1]

{'example_id': '572502',
 'text': "User: At September 30 , 2019 , we had $ 1.5 billion available on our revolving credit facility .\nAssistant Prediction:\n{'LineOfCreditFacilityRemainingBorrowingCapacity': ['1.5']}"}

In [12]:
gold_labels_list = []

for example in val_data:
    text = example['text']
    parts = text.split("Assistant Prediction:")
    user = parts[0][6:].strip() # Also removes "User: " prefix
    prediction = parts[1].strip()

    print("User Input:", user)
    print("Model Prediction:", prediction)

    if prediction == "No XBRL associated data.":
        gold_labels = {}
    else:
        gold_labels = json.loads(prediction.replace("'", '"'))

    gold_labels_list.append(gold_labels)

User Input: The Trust has entered into a pay - fixed receive - variable interest rate swap , fixing the LIBOR component of this rate at 1.07 % .
Model Prediction: {'DerivativeFixedInterestRate': ['1.07']}
User Input: At September 30 , 2019 , we had $ 1.5 billion available on our revolving credit facility .
Model Prediction: {'LineOfCreditFacilityRemainingBorrowingCapacity': ['1.5']}
User Input: The other components of net periodic pension cost and net periodic postretirement benefit cost are required to be presented in the income statement separately from the service cost component and outside a subtotal of income from operations , if one is presented .
Model Prediction: No XBRL associated data.
User Input: See also Note 11 , Commitments and Contingencies .
Model Prediction: No XBRL associated data.
User Input: 10 Table of Contents ECHOSTAR CORPORATION NOTES TO CONDENSED CONSOLIDATED FINANCIAL STATEMENTS - Continued ( Unaudited ) Note 3 .
Model Prediction: No XBRL associated data.
User

In [13]:
gold_labels_list[:5]

[{'DerivativeFixedInterestRate': ['1.07']},
 {'LineOfCreditFacilityRemainingBorrowingCapacity': ['1.5']},
 {},
 {},
 {}]

In [14]:
def compare_json_objects(gold, pred):
    """
    Compares two JSON objects (dictionaries) and calculates the f1 score.

    Args:
        gold: Dict containing the gold labels.
        pred: Dict containing the predicted labels.
    Returns:
        float: f1 score
    """
    
    if not gold and not pred:
        return 1.0 # perfect match if both are empty
    if not gold or not pred:
        return 0.0

    tp = 0
    fp = 0
    fn = 0
    
    for key, gold_values in gold.items():

        pred_values = pred.get(key, []) # default to empty list if key not found

        for value in gold_values:
            if value in pred_values:
                tp += 1 # correctly predicted
            else:
                fn += 1 # in gold but not predicted

        fp += len(set(pred_values) - set(gold_values)) # predicted but not in gold
            
    extra_keys = set(pred.keys()) - set(gold.keys()) # Keys in pred but not in gold

    for key in extra_keys:
        # all values in these keys are incorrect
        pred_values = pred[key]
        fp += len(pred_values)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return f1

In [15]:
preds = [{'DerivativeFixedInterestRate': ['1.07', '1.08']}, {'Liabilities': ['500,000']}, {}]

In [16]:
scores = []

for pred, gold in zip(preds, gold_labels_list):
    score = compare_json_objects(gold, pred)
    scores.append(score)

print(scores)

avg_score = sum(scores) / len(scores)
avg_score

[0.6666666666666666, 0.0, 1.0]


0.5555555555555555